In [4]:
import numpy as np 
import matplotlib.pyplot as plt 
import random

In [8]:
def W(u):
    
    output = np.zeros(u.shape)
    for index,val in enumerate(u):
        
        if val >= 100:
            output[index] = val - np.log(val)
    
        if val < 100 and val > - 20 :
            w = 1
            v = 0
            while abs(w - v)/abs(w) > 1.e-8:
                v = w
                e = np.exp(w)
                f = w*e - val
                w = w - f/((e*(w+1) - (w+2)*f/(2*w+2)))
            output[index] = w
            
        if val <=-20:
            output[index] = 0
        
    return output

In [11]:
W(np.array([100,2,0.5,-50]))

array([95.39482981,  0.8526055 ,  0.35173371,  0.        ])

In [ ]:
beta_grid = np.exp(np.log(10)*np.linspace(-4,2,7))

def proximal_gradient_entr(lamb,K,y,T,xmin,xmax,display = False):
    tol = 1e-7
    max_iter = 1000
    nb_iter = 0
    m = K.shape[0]
    n = K.shape[1]
    u = np.zeros((n,1))
    w,v = np.linalg.eig(np.dot(K.T,K) + lamb * np.dot(D.T,D))
    L = max(w)
    epsilon = 1/L
    norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
    normD = np.linalg.norm(np.dot(D,u))
    obj = 0.5 * norm2 **2 + 0.5 * lamb * normD ** 2 + ind(u,xmin,xmax)
    last_obj = 0
    if display:
        print("Objectif : {}, ||Ku-y||^2 : {}, ||Du||^2 : {}, ind : {}".format(obj,norm2**2,normD**2,ind(u,xmin,xmax)))
    
    while (abs(obj - last_obj) > tol and nb_iter < max_iter):
        w = u - epsilon * (np.dot(K.T,np.dot(K,u) - y.reshape(m,1)) + lamb * np.dot(D.T,np.dot(D,u)))
        last_obj = obj
        u = prox_cons(w,xmin,xmax) 
        norm2 = np.linalg.norm(np.dot(K,u) - y.reshape(m,1))
        normD = np.linalg.norm(np.dot(D,u))
        obj = 0.5 * norm2 **2 + 0.5*lamb * normD ** 2 + ind(u,xmin,xmax)
        if display:
            print("Objectif : {}, ||Ku-y||^2 : {}, ||Du||^2 : {}, ind : {}".format(obj,norm2**2,normD**2,ind(u,xmin,xmax)))
        nb_iter += 1
        
    return u
        
def prox_entr(w,xmin,xmax):
    m = w.shape[0]
    out = np.zeros((m,1))
    for index in range(m):
        value = w[index,0]
        if value >  xmax:
            out[index,0] = xmax
        elif value <  xmin:
            out[index,0] = xmin
        elif value >= xmin and value <= xmax:
            out[index,0] = value
    return out

def ind(x,xmin,xmax):
    n = x.shape[0]
    for i in range(n):
        if x[i] > xmax or x[i] < xmin:
            return 1e30
    return 0

for i in range(beta_grid.shape[0]):
    beta = beta_grid[i]
    x_hat = proximal_gradient_smooth_cons(beta,K,y,T,xmin,xmax) #,True
    print("MSE : {}, beta : {}\n".format(normalized_MSE(x_hat,x.reshape(200,1)),beta))
    plt.plot(np.log(T),x_hat)
    plt.plot(np.log(T),x)
    plt.grid()
    plt.show()